# AlpesHearth

## Caso de estudio AlpesHearth. 

El jupyter notebook aqui presente se basa en el caso de estudio AlpesHearth, el cual tiene como objetivos:
- Aplicar técnicas de regresión para construir un modelo predictivo que permita estimar el riesgo cardiovascular de una persona siguiendo el ciclo de machine learning.
- Determinar los principales factores de riesgo cardiovascular con base en los datos.
- Aplicar y comprender un modelo de regresión lineal.
- Reconocer posibles sesgos del modelo de aprendizaje de máquina.
- Comunicar de forma clara y sintética los resultados obtenidos.

Se utilizaron las siguientes librerias de Python para el procesamiento y analisis de datos: 
- Pandas
- Scikit-Learn
- Matplotlib, Seaborn

#### Instalar las librerias necesarias

In [ ]:
%pip install pandas numpy matplotlib seaborn ipython scikit-learn

#### Importar las librerias

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
from IPython.display import display
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [ ]:
from importlib.metadata import version
print(f"Versión de Pandas: {version('pandas')}")
print(f"Versión de Numpy: {version('numpy')}")
print(f"Versión de Matplotlib: {version('matplotlib')}")
print(f"Versión de Seaborn: {version('seaborn')}")

### Exploración de los datos

In [ ]:
datos_model = pd.read_csv('./Datos/Datos Lab 1.csv')

In [ ]:
data = datos_model.copy()

In [ ]:
data.head()

Aqui podemos ver como hay columnas de datos repetidos y/o muy parecidos. Por ejemplo, Height (m) y Height (cm), Waist-to-Height Ratio Height y Abdominal Circumference, BMI Weight y Height etc.

A la hora de realizar el analisis estadistico, sera necesario ver cuales de estas variables son redundantes y no nos aportan al modelo. Asimismo, hay que revisar para casos tales como las dos estaturas, si coinciden. De no coincidir, habra que realizar un promedio o utilizar alguna otra tecnica.

In [ ]:
display(data.sample(5))

In [ ]:
data.shape

Esto nos dice que el conjunto de datos esta compuesto por 1.639 registros (filas) y 24 variables (columnas). 

In [ ]:
data.info() 

Arriba podemos ver el detalle de cada columna, como el nombre, el número de valores no nulos (en este caso al tener 1.639, indica que no se tienen valores faltantes) y el tipo de los datos de cada columna, al igual que el número de columnas con cada tipo.

Es interesante ver que las columnas que no son numéricas (int o float) son `Patient ID`, `Date of Service`, `Sex`, `Blood Pressure (mmHg)`, `Smoking Status`, `Diabetes Status`, `Physical Activity Level`, `Family History of CVD`, y `Blood Pressure Category`, las cuales están en formato object, esto es algo importante y que debemos tener en cuenta. 

Como sabemos que `Blood Pressure Category` = `Systolic BP` / `Diastolic BP`, habra que reemplazar los NULLs por los respectivos valores de `Blood Pressure Category`. Mas adelante se revisara si estos valores son redundantes y si sera necesario remover alguna de estas columnas.

Un insumo imprescindible para entender un conjunto de datos es el diccionario, ya que este nos permite conocer el significado de cada variable y sus rangos válidos.  

Al mismo tiempo, al revisar los valores non-null de cada una de las columnas, podemos ver que hay muchos datos NULL dentro de la base de datos. Por lo que se debera realizar algo al respecto.

In [ ]:
cols_con_null = []

for col in data.columns:
    if data[col].count() != len(data):
        cols_con_null.append(col)

cols_con_null

In [ ]:
diccionario = pd.read_excel('./Datos/DiccPacientes.xlsx')
pd.set_option('display.max_colwidth', None)
diccionario

Con el fin de enriquecer el modelo, se revisaran las siguientes variables mas adelante:

Derivadas de datos existentes
- Grupo etario
- Categoria de BMI
- Categoria de glucosa
- Ratio colesterol
- Pulse Pressure
- Mean Arterial Pressure

Fuentes externas
- Mes
- Dias de la semana
- Estacion del anio

Scores clinicos establecidos
- Framingham Risk Score
- ASCVD Risk Score

Interacciones
- Fumador + Diabetes
- BMI * Actividad Fisica
- Edad * Presion Arterial

In [ ]:
data.describe()

Para el peso seria prudente revisar si es posible? Es decir si se podria un individuo de 13kg con 20 anios? 

Un Estimated LDL de -92.05 es fisiologicamente imposible. Por lo que es posible que este valor no sea real. 

Un valor negativo de CVD Risk Score no es posible. Los valores posibles son entre 0 y 100. Por lo que hay errores en los minimos y maximos.

Aqui podemos realizar varias tecnicas de normalizacion con el fin de mejorar la habilidad predictiva del modelo.

In [ ]:
# Revisar formatos, asegurarse que no hayan representaciones diferentes de los mismos datos...

objetos = ['Patient ID', 'Date of Service', 'Sex', 
           'Blood Pressure (mmHg)', 'Smoking Status', 
           'Diabetes Status', 'Physical Activity Level', 
           'Family History of CVD', 'Blood Pressure Category', 
           'CVD Risk Level'
] 

for i in objetos:
    display(data[i].value_counts())

Aqui podemos ver tanto que hay IDs repetidos, como que hay formatos diferentes para las fechas de servicio, por lo que sera necesario revisar si es que hay filas de datos duplicadas, y refactorizar las fechas a un formato estandar. 

In [ ]:
display(data.duplicated().sum())

display(data[data.duplicated(keep=False)])

Aqui vemos las filas de datos que se encuentran completamente duplicadas. Estas se removeran en la siguiente fase del proyecto.

#### Gráficas de las variables

In [ ]:
# Se incluyen unicamente variables numericas para los boxplots
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns

fig, axes = plt.subplots(4, 4, figsize=(16, 12))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    if i < len(axes):
        data.boxplot(column=col, ax=axes[i])
        axes[i].set_title(col)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(data[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.show()

In [ ]:
# Aqui se quitan los rubros 'Patient ID' y 'Date of Service'
categoricas = objetos[2:]

fig, axes = plt.subplots(3, 3, figsize=(14, 10))
axes = axes.flatten()

for i, col in enumerate(categoricas):
    if i < len(axes):
        data[col].value_counts().plot(kind='bar', ax=axes[i])
        axes[i].set_title(col)
        axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
data_encoded = data.copy()

le = LabelEncoder()
for col in categoricas:
    data_encoded[col] = le.fit_transform(data_encoded[col])

# Correlacion con CVD Risk Score
correlaciones = data_encoded[categoricas + ['CVD Risk Score']].corr()['CVD Risk Score'].drop('CVD Risk Score').sort_values(ascending=False)

print(correlaciones)

plt.figure(figsize=(10, 5))
correlaciones.plot(kind='bar', color=['green' if x > 0 else 'red' for x in correlaciones])
plt.title('Correlación de variables categóricas con CVD Risk Score')
plt.ylabel('Correlación')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

#### Hallazgos de la exploración:

##### Valores imposibles detectados:
- Estimated LDL negativo (-92): fisiológicamente imposible
- CVD Risk Score negativo y >100: fuera de rango válido [0-100]
- Total Cholesterol ≈ 0: imposible
- HDL < 10: imposible
- BMI < 7: menor al mínimo registrado (~7.5)

##### Outliers extremos pero posibles:
- Age ~10: inusual para datos cardiovasculares
- BMI > 50: obesidad mórbida extrema
- CVD Risk Score > 60: pacientes de alto riesgo

##### Decisiones para preparación:
- Convertir valores imposibles a NaN
- Imputar con media del train después del split

##### Multicolinealidad
- Height (m) y Height (cm): 0.92 se debe eliminar una
- Total Cholesterol y Estimated LDL: 0.85 se deberia considerar eliminar una.
- Abdominal Circumference y Waist-to-Height ratio: 0.84 se deberia considerar eliminar una.
- Weight y BMI: 0.58 relacion a tener en cuenta.
- Blood Pressure = Systolic/Diastolic

##### Correlaciones con CVD Risk Score (Numericas)
- BMI: 0.13 la mas alta, pero debil.
- Systolic BP: 0.09
- El resto muy bajas ~0.

##### Correlaciones con CVD Risk Score (Categoricas)
- Diabetes Status: 0.15 es el mejor predictor
- Blood Pressure Category: -0.07
- Family History of CVD: -0.04
- CVD Risk Level: ~0 raro porque deberia derivarse del CVD Risk Score

### Preparación de datos

In [ ]:
data = data.drop_duplicates()

Aqui se quitan las filas de datos duplicados.

In [ ]:
data = data.copy()
data["Date of Service"] = pd.to_datetime(data["Date of Service"], format='mixed')
data
# TODO: Como se usa formato 'mixed', para datos como: 08-09-2022, se adivina cual es el mes y cual es el dia.

Aqui se realiza la respectiva refactorizacion de las fechas.

In [ ]:
# Convertir valores imposibles a NaN para luego imputar

# Estimated LDL: no puede ser negativo, rango 0 - 300
data.loc[data['Estimated LDL (mg/dL)'] < 0, 'Estimated LDL (mg/dL)'] = np.nan
data.loc[data['Estimated LDL (mg/dL)'] > 300, 'Estimated LDL (mg/dL)'] = np.nan

# CVD Risk Score: rango 0 - 100
data.loc[data['CVD Risk Score'] < 0, 'CVD Risk Score'] = np.nan
data.loc[data['CVD Risk Score'] > 100, 'CVD Risk Score'] = np.nan

# Total Cholesterol: no puede ser negativo
data.loc[data['Total Cholesterol (mg/dL)'] < 0, 'Total Cholesterol (mg/dL)'] = np.nan

# BMI: El valor minimo reportado en un ser viviente es de ~7.5
data.loc[data['BMI'] < 7, 'BMI'] = np.nan

Se clippearon valores imposibles de la base de datos.

In [ ]:
# Conversion a m
height_m = data["Height (m)"]
height_cm_to_m = data["Height (cm)"] / 100

# Promedio donde ambos existen, sino el que existe
data.loc[:,"Height (m)"] = (height_m.fillna(height_cm_to_m) + height_cm_to_m.fillna(height_m)) / 2

data = data.drop(columns=["Height (cm)"])

Para el parametro height, se realizo: 
- Si existian tanto Height (m), como Height (cm), se calculo un promedio.
- Si solo existia uno de estos se dejo este.
- Si no existia se tomo como el promedio global de las estaturas.

In [ ]:
BMI = (data["Weight (kg)"] / data["Height (m)"] ** 2)
data["BMI"] = data["BMI"].fillna(BMI)

Para los datos que no tenian BMI, se utilizo la formula del BMI para agregarla.
BMI = weight (kg) / height (m) ^2

In [ ]:
bp_split = data['Blood Pressure (mmHg)'].str.split('/', expand=True).astype(float)
data['Systolic BP'] = data['Systolic BP'].fillna(bp_split[0])
data['Diastolic BP'] = data['Diastolic BP'].fillna(bp_split[1])

Aqui llenamos los datos restantes de `Systolic BP` y `Diastolic BP` con los respectivos datos que tienen en `Blood Pressure (mmHg)`.

In [ ]:
WHtR = data['Abdominal Circumference (cm)'] / (data['Height (m)'] * 100)
data['Waist-to-Height Ratio'] = data['Waist-to-Height Ratio'].fillna(WHtR)

# Verificar rangos
print(data['Waist-to-Height Ratio'].describe())

Si bien es cierto que waist circumference es diferente de abdominal circumference, sigue siendo una aproximacion valida y suelen estar linealmente correlacionadas. Por lo que, para las filas de datos en las que faltan valores para el waist to height ratio, se decidio utilizar el rubro `Abdominal Circumference (cm)` y dividirlo entre el rubro de `Height (m)` * 100.

In [ ]:
abdominal_circum = data['Height (m)'] * data['Waist-to-Height Ratio']/100
data['Abdominal Circumference (cm)'] = data['Abdominal Circumference (cm)'].fillna(abdominal_circum)

weight = data['BMI'] * (data['Height (m)'] ** 2)
data['Weight (kg)'] = data['Weight (kg)'].fillna(weight)

estimated_LDL = data['Total Cholesterol (mg/dL)'] - data['HDL (mg/dL)'] - 30
data['Estimated LDL (mg/dL)'] = data['Estimated LDL (mg/dL)'].fillna(estimated_LDL)

tc_calc = data['Estimated LDL (mg/dL)'] + data['HDL (mg/dL)'] + 30
data['Total Cholesterol (mg/dL)'] = data['Total Cholesterol (mg/dL)'].fillna(tc_calc)

hdl_calc = data['Total Cholesterol (mg/dL)'] - data['Estimated LDL (mg/dL)'] - 30
data['HDL (mg/dL)'] = data['HDL (mg/dL)'].fillna(hdl_calc)

Aqui, utilizamos los datos que ya tenemos para calcular algunos de los que faltan. Esto se pudo realizar para los rubros: 
- `Abdominal Circumference (cm)`
- `Weight (kg)`
- `Estimated LDL (mg/dL)`
- `Total Cholesterol (mg/dL)`
- `HDL (mg/dL)`

Sin embargo, siguen faltando algunos datos. 

In [ ]:
data.info() 

#### Ingeniería de características

Continuando la fase de preparacion de datos, se va a realizar una agregacion de features que podrian ser importantes para revisar cuales de estas podrian enriquecer el modelo.

Las variables que se tomaran en cuenta para esta parte son: 
- Transformaciones de fila: Grupo etario, Categoria BMI, Categoria glucosa
- Formulas con datos de la misma fila: Ratio colesterol, Pulse Pressure, MAP
- Derivadas: Mes, Dia de semana, Estacion
- Interacciones: Fumador + Diabetes, BMI x Actividad
- Formulas Establecidas: Framingham, ASCVD

In [ ]:
def framingham_score(row):
    score = 0
    
    # Edad
    if row['Age'] >= 55: score += 3
    elif row['Age'] >= 45: score += 2
    elif row['Age'] >= 35: score += 1
    
    # Sexo (hombres tienen mayor riesgo)
    if row['Sex'] == 'M': score += 1
    
    # Colesterol Total
    if row['Total Cholesterol (mg/dL)'] >= 280: score += 3
    elif row['Total Cholesterol (mg/dL)'] >= 240: score += 2
    elif row['Total Cholesterol (mg/dL)'] >= 200: score += 1
    
    # HDL (protector, menor = peor)
    if row['HDL (mg/dL)'] < 35: score += 2
    elif row['HDL (mg/dL)'] < 45: score += 1
    elif row['HDL (mg/dL)'] >= 60: score -= 1
    
    # Presión sistólica
    if row['Systolic BP'] >= 160: score += 3
    elif row['Systolic BP'] >= 140: score += 2
    elif row['Systolic BP'] >= 130: score += 1
    
    # Fumador
    if row['Smoking Status'] == 'Y': score += 2
    
    # Diabetes
    if row['Diabetes Status'] == 'Y': score += 2
    
    return score


def ascvd_score(row):
    score = 0
    
    # Edad (mas peso)
    score += row['Age'] * 0.5
    
    # Sexo 
    if row['Sex'] == 'M': score += 5
    
    # Colesterol
    score += row['Total Cholesterol (mg/dL)'] * 0.02
    score -= row['HDL (mg/dL)'] * 0.05
    
    # Presion
    score += row['Systolic BP'] * 0.03
    
    # Factores binarios
    if row['Smoking Status'] == 'Y': score += 7
    if row['Diabetes Status'] == 'Y': score += 5
    
    return score


def grupo_etario(age):
    if age < 30: return 'Joven'
    elif age < 45: return 'Adulto'
    elif age < 60: return 'Adulto Mayor'
    else: return 'Senior'
    
    
def categoria_bmi(bmi):
    if bmi < 18.5: return 'Bajo peso'
    elif bmi < 25: return 'Normal'
    elif bmi < 30: return 'Sobrepeso'
    else: return 'Obesidad'
    
    
def categoria_glucosa(fbs):
    if fbs < 100: return 'Normal'
    elif fbs < 126: return 'Prediabetes'
    else: return 'Diabetes'

In [ ]:
# Derivadas simples
data['Pulse Pressure'] = data['Systolic BP'] - data['Diastolic BP']
data['MAP'] = data['Diastolic BP'] + (data['Pulse Pressure'] / 3)
data['Cholesterol Ratio'] = data['Total Cholesterol (mg/dL)'] / data['HDL (mg/dL)'] # Queda con unos cuantos valores NaN

# Temporales
data['Month'] = data['Date of Service'].dt.month
data['DayOfWeek'] = data['Date of Service'].dt.dayofweek
data['Year'] = data['Date of Service'].dt.year

# Interacciones
data['Smoker_Diabetic'] = (data['Smoking Status'] == 'Y') & (data['Diabetes Status'] == 'Y')

# BMI x Actividad
activity_risk = {'Low': 3, 'Moderate': 2, 'High': 1}
data['Activity Risk'] = data['Physical Activity Level'].map(activity_risk)

Estas variables podemos agregarlas a la base de datos antes de realizar el split entre training y testing. 

Sin embargo, algunas de las variables de interes que se quieren evaluar se realizaran despues del split.

#### Split
Ahora, realizamos el split de los datos con una semilla de 42 (random_state) y un porcentaje de 25% para el tamaño del conjunto de prueba (test_size=0.25).

In [ ]:
exclude_cols = ['CVD Risk Score', 'CVD Risk Level', 'Patient ID', 
                'Date of Service', 'Blood Pressure (mmHg)']

X = data.drop(columns=exclude_cols) 
Y = data['CVD Risk Score']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

#### Imputacion y Agregacion de 6 Features extras

In [ ]:
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()

imputer_cat = SimpleImputer(strategy='most_frequent')
X_train[categorical_cols] = imputer_cat.fit_transform(X_train[categorical_cols])
X_test[categorical_cols] = imputer_cat.transform(X_test[categorical_cols])

imputer_num = SimpleImputer(strategy='mean')
X_train[numerical_cols] = imputer_num.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = imputer_num.transform(X_test[numerical_cols])

# Creacion de features X_train
X_train['Age Group'] = X_train['Age'].apply(grupo_etario)
X_train['BMI Category'] = X_train['BMI'].apply(categoria_bmi)
X_train['Glucose Category'] = X_train['Fasting Blood Sugar (mg/dL)'].apply(categoria_glucosa)
X_train['BMI x Sedentarism'] = X_train['BMI'] * X_train['Activity Risk']
X_train['ASCVD Score'] = X_train.apply(ascvd_score, axis=1)
X_train['Framingham Score'] = X_train.apply(framingham_score, axis=1)

# Creacion de features X_test
X_test['Age Group'] = X_test['Age'].apply(grupo_etario)
X_test['BMI Category'] = X_test['BMI'].apply(categoria_bmi)
X_test['Glucose Category'] = X_test['Fasting Blood Sugar (mg/dL)'].apply(categoria_glucosa)
X_test['BMI x Sedentarism'] = X_test['BMI'] * X_test['Activity Risk']
X_test['ASCVD Score'] = X_test.apply(ascvd_score, axis=1)
X_test['Framingham Score'] = X_test.apply(framingham_score, axis=1)

### Construcción de un modelo de regresión lineal

In [ ]:
datos_prueba = pd.read_csv(
    './Datos/Datos Test Lab 1.csv',
    sep=';'
)